In [72]:
import importlib
from rl import multi_stock_env
importlib.reload(multi_stock_env)
MultiStockEnv = multi_stock_env.MultiStockEnv

In [4]:
from pprint import pprint
from pathlib import Path
import pandas as pd

In [5]:
MARKET = "csi300"
BENCHMARK = "SH000300"

EXP_NAME = "tutorial_exp"

data_provide_uri = "~/qlib_data/cn_data"

# Data

In [6]:
import qlib

qlib.init(provider_uri=data_provide_uri, region="cn")

[1135:MainThread](2025-08-20 22:22:14,903) INFO - qlib.Initialization - [config.py:452] - default_conf: client.
[1135:MainThread](2025-08-20 22:22:15,480) INFO - qlib.Initialization - [__init__.py:79] - qlib successfully initialized based on client settings.
[1135:MainThread](2025-08-20 22:22:15,481) INFO - qlib.Initialization - [__init__.py:81] - data_path={'__DEFAULT_FREQ': PosixPath('/root/qlib_data/cn_data')}


In [ ]:
import qlib
from qlib.data import D
import pandas as pd
from stable_baselines3 import PPO
  # 你需要自定义这个环境

# 初始化 Qlib
qlib.init(provider_uri=data_provide_uri, region="cn")
# 获取 csi300 成分股数据
instruments = D.list_instruments(D.instruments("csi300"))

results_df = {}
for instrument in list(instruments)[:10]:  # 只获取前10个股票数据，避免过多请求
    # 获取单个股票数据
    df = D.features(
        [instrument],
        ["$open", "$high", "$low", "$close", "$factor", "$adjclose", "$volume", "$change", "$amount"],
        start_time="2020-01-01",
        end_time="2025-08-31",
    )
    
    # 处理数据
    df = df.dropna()
    df = df.reset_index(drop=True)
    df.columns = [col.replace("$", "") for col in df.columns]

    if not df.empty:
        results_df[instrument] = df
    # 对 volume 做 Min-Max 归一化
    df["volume_norm"] = (df["volume"] - df["volume"].min()) / (df["volume"].max() - df["volume"].min())
    # 

#df = D.features(instruments, ["$close", "$open", "$high", "$low", "$volume"], start_time="2020-01-01", end_time="2025-12-31")



[91441:MainThread](2025-08-19 23:26:25,492) INFO - qlib.Initialization - [config.py:452] - default_conf: client.
[91441:MainThread](2025-08-19 23:26:25,494) INFO - qlib.Initialization - [__init__.py:79] - qlib successfully initialized based on client settings.
[91441:MainThread](2025-08-19 23:26:25,495) INFO - qlib.Initialization - [__init__.py:81] - data_path={'__DEFAULT_FREQ': PosixPath('/root/qlib_data/cn_data')}


In [44]:
from qlib.data import D
df_native = D.features(
        D.instruments(MARKET),
        ["$open", "$high", "$low", "$close", "$factor", "$adjclose", "$volume", "$change", "$amount"],
        start_time="2008-01-01",
        end_time="2025-08-31",
    )
df_native.tail(10)

$open     $high      $low    $close   $factor  \
instrument datetime                                                       
SZ302132   2025-08-01  7.471598  7.471598  7.283923  7.285693  0.088526   
           2025-08-04  7.260995  7.480542  7.259224  7.473460  0.088527   
           2025-08-05  7.472465  7.613221  7.420235  7.505220  0.088526   
           2025-08-06  7.455738  8.177232  7.452197  7.862961  0.088527   
           2025-08-07  7.773579  7.832892  7.587672  7.640788  0.088527   
           2025-08-08  7.613321  7.665552  7.515941  7.519483  0.088527   
           2025-08-11  7.505260  7.620345  7.498178  7.536245  0.088526   
           2025-08-12  7.524734  7.681426  7.448602  7.579621  0.088526   
           2025-08-13  7.529137  7.632712  7.521169  7.586679  0.088526   
           2025-08-14  7.576179  7.576179  7.436306  7.465520  0.088527   

                        $adjclose      $volume   $change       $amount  
instrument datetime                                                     
SZ302132   2025-08-01  495.500000  1444457.375 -0.027876  1.062855e+06  
           2025-08-04  508.269989  1326121.250  0.025759  9.851958e+05  
           2025-08-05  510.429993  1428743.875  0.004264  1.075837e+06  
           2025-08-06  534.760010  3891991.250  0.047653  3.045948e+06  
           2025-08-07  519.650024  2343532.000 -0.028259  1.793617e+06  
           2025-08-08  511.399994  1350687.750 -0.015873  1.023233e+06  
           2025-08-11  512.539978  1247831.250  0.002237  9.437501e+05  
           2025-08-12  515.489990  1810458.875  0.005756  1.366656e+06  
           2025-08-13  515.969971  1514339.500  0.000934  1.147399e+06  
           2025-08-14  507.730011  1372873.000 -0.015986  1.026918e+06

In [27]:
results_df[next(iter(results_df))]

,open,high,low,close,factor,adjclose,volume,change,amount
0,11.144468,11.345270,11.077535,11.291723,0.669337,1841.680054,2.286189e+06,0.025532,2571196.500
1,11.338562,11.586217,11.325175,11.499203,0.669337,1875.520020,1.667614e+06,0.018376,1914495.500
2,11.385407,11.606288,11.318474,11.425567,0.669336,1863.510010,1.287968e+06,-0.006403,1477930.250
3,11.465767,11.566168,11.345286,11.479154,0.669338,1872.250000,1.088549e+06,0.004687,1247047.125
4,11.378771,11.412238,11.131115,11.151196,0.669339,1818.760010,1.266658e+06,-0.028571,1423608.750
...,...,...,...,...,...,...,...,...,...
1357,10.084078,10.124478,10.011355,10.019436,0.808019,1634.170044,1.026951e+06,-0.005613,1032586.750
1358,10.019428,10.035589,9.914386,9.938626,0.808018,1620.989990,1.156351e+06,-0.008065,1150305.125
1359,9.938604,10.019406,9.938604,9.962845,0.808017,1624.939941,8.416364e+05,0.002439,839635.625
1360,9.995197,10.019438,9.882074,9.906315,0.808019,1615.719971,1.385874e+06,-0.005677,1376777.875


In [ ]:
from rl.multi_stock_env import MultiStockEnv

In [ ]:
# 构建 RL 环境
env = MultiStockEnv(results_df, list(results_df))

# 训练 RL agent
model = PPO("MlpPolicy", env, verbose=1)
model.learn(total_timesteps=100000)


In [ ]:

# 回测
obs = env.reset()
done = False
while not done:
    action, _states = model.predict(obs)
    obs, reward, done, info = env.step(action)

# 评估策略表现
env.render()

In [23]:
## TODO
## 需要对齐股价数据，保证每一行是同个时间点
## 用factor还原实际交易的价格
## 训练集和验证集要分开在两个时间段，做回测

In [31]:
handler_kwargs = {
    "start_time": "2008-01-01",
    "end_time": "2025-08-31",
    "fit_start_time": "2008-01-01",
    "fit_end_time": "2014-12-31",
    "instruments": MARKET,
}
handler_conf = {
    "class": "Alpha158",
    "module_path": "qlib.contrib.data.handler",
    "kwargs": handler_kwargs,
}
from qlib.utils import init_instance_by_config
hd = init_instance_by_config(handler_conf)


[1135:MainThread](2025-08-20 22:39:16,110) INFO - qlib.timer - [log.py:127] - Time cost: 13.225s | Loading data Done
[1135:MainThread](2025-08-20 22:39:17,046) INFO - qlib.timer - [log.py:127] - Time cost: 0.208s | DropnaLabel Done
[1135:MainThread](2025-08-20 22:39:19,043) INFO - qlib.timer - [log.py:127] - Time cost: 1.996s | CSZScoreNorm Done
[1135:MainThread](2025-08-20 22:39:19,045) INFO - qlib.timer - [log.py:127] - Time cost: 2.934s | fit & process data Done
[1135:MainThread](2025-08-20 22:39:19,046) INFO - qlib.timer - [log.py:127] - Time cost: 16.162s | Init data Done


In [45]:
df = hd.fetch()
# 假设 df 是 MultiIndex ['datetime', 'instrument']
all_dates = set(df.index.get_level_values('datetime'))
instruments = df.index.get_level_values('instrument').unique()

valid_stocks = []
for inst in instruments:
    inst_dates = set(df.loc[(slice(None), inst), :].index.get_level_values('datetime'))
    if inst_dates == all_dates:
        valid_stocks.append(inst)

print(valid_stocks)

['SH600000', 'SH600009', 'SH600010', 'SH600015', 'SH600016', 'SH600019', 'SH600028', 'SH600029', 'SH600030', 'SH600031', 'SH600036', 'SH600048', 'SH600050', 'SH600085', 'SH600104', 'SH600111', 'SH600196', 'SH600309', 'SH600362', 'SH600519', 'SH600547', 'SH600585', 'SH600660', 'SH600690', 'SH600741', 'SH600795', 'SH600900', 'SH601006', 'SH601009', 'SH601111', 'SH601166', 'SH601169', 'SH601318', 'SH601328', 'SH601398', 'SH601600', 'SH601628', 'SH601857', 'SH601988', 'SH601998', 'SZ000001', 'SZ000002', 'SZ000063', 'SZ000157', 'SZ000338', 'SZ000425', 'SZ000538', 'SZ000568', 'SZ000625', 'SZ000651', 'SZ000768', 'SZ000858', 'SZ000876', 'SZ000895', 'SZ002142']


In [48]:
# 假设 valid_stocks 是你筛选出来的股票列表
df_filtered = df[df.index.get_level_values('instrument').isin(valid_stocks)]

df_merged = df_filtered.join(df_native[['$adjclose']], how='left')

In [55]:
df_merged = df_merged[~df_merged['$adjclose'].isna()]

In [74]:
# 假设 df 是 Alpha158 处理后的 DataFrame，索引为 ['datetime', 'instrument']
df_merged

# 划分训练集和验证集
train_mask = (df_merged.index.get_level_values('datetime') >= '2020-01-01') & (df_merged.index.get_level_values('datetime') <= '2023-12-31')
valid_mask = (df_merged.index.get_level_values('datetime') >= '2024-01-01') & (df_merged.index.get_level_values('datetime') <= '2025-08-31')
train_df = df_merged[train_mask]
valid_df = df_merged[valid_mask]

# 转为 dict 格式
def df_to_dict(long_df):
    df_dict = {}
    for inst, subdf in long_df.groupby('instrument'):
        subdf = subdf.sort_index(level='datetime')
        subdf = subdf.reset_index()
        df_dict[inst] = subdf
    return df_dict

train_dict = df_to_dict(train_df)
valid_dict = df_to_dict(valid_df)
tickers = list(train_dict.keys())
tickers_valid = list(valid_dict.keys())

# 传入 RL 环境
env_train = MultiStockEnv(train_dict, tickers)
env_valid = MultiStockEnv(valid_dict, tickers_valid)

In [73]:
from stable_baselines3 import PPO
import torch

torch.set_default_dtype(torch.float32)
# 训练
model = PPO(
    "MlpPolicy",
    env_train,
    learning_rate=3e-4,
    n_steps=2048,
    batch_size=1024,
    gamma=0.99,
    gae_lambda=0.95,
    clip_range=0.2,
    ent_coef=0.01,
    vf_coef=0.5,
    max_grad_norm=0.5,
    verbose=1,
    policy_kwargs=dict(net_arch=[256, 256])
)
model.learn(total_timesteps=1_000_000,
            )

Using cuda device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


/opt/software/miniconda3/envs/quant/lib/python3.13/site-packages/stable_baselines3/common/on_policy_algorithm.py:150: UserWarning: You are trying to run PPO on the GPU, but it is primarily intended to run on the CPU when not using a CNN policy (you are using ActorCriticPolicy which should be a MlpPolicy). See https://github.com/DLR-RM/stable-baselines3/issues/1245 for more info. You can pass `device='cpu'` or `export CUDA_VISIBLE_DEVICES=` to force using the CPU.Note: The model will train, but the GPU utilization will be poor and the training might take longer than on CPU.
  warnings.warn(


---------------------------------
| rollout/           |          |
|    ep_len_mean     | 503      |
|    ep_rew_mean     | -0.557   |
| time/              |          |
|    fps             | 69       |
|    iterations      | 1        |
|    time_elapsed    | 58       |
|    total_timesteps | 4096     |
---------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 475         |
|    ep_rew_mean          | -0.536      |
| time/                   |             |
|    fps                  | 64          |
|    iterations           | 2           |
|    time_elapsed         | 127         |
|    total_timesteps      | 8192        |
| train/                  |             |
|    approx_kl            | 0.009597785 |
|    clip_fraction        | 0.256       |
|    clip_range           | 0.2         |
|    entropy_loss         | -78.1       |
|    explained_variance   | 0.56        |
|    learning_rate        | 0.

KeyboardInterrupt: 